In [1]:
import torch
import monai
import dataloader
import os
import pandas as pd
import numpy as np
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
from torch.nn import MSELoss
from torch.nn import BCEWithLogitsLoss
import numpy as np
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import DataLoader
import torch.optim.lr_scheduler
from monai.losses import DiceLoss
from monai.losses import FocalLoss
from monai.networks.nets import UNet
import torchmetrics
from torchmetrics.classification import BinaryPrecision, BinaryRecall
import math

In [2]:
data_path = '../data/hm30rad/'
df = pd.read_csv('../data/train_labels.csv')
names = df['tomo_id'].astype(str).unique().tolist()

In [3]:
import optuna
import torch
from torch.utils.data import DataLoader
from torch.optim import Adam
from monai.losses import DiceLoss
from monai.networks.nets import UNet
from optuna.pruners import MedianPruner
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Define the objective function for Optuna
def objective(trial):
    # ------------------------------ #
    #        HYPERPARAMETERS         #
    # ------------------------------ #
    lr = trial.suggest_float("lr", 5e-8, 5e-4, log=True)
    decay = 0.9
    # decay = trial.suggest_float('decay', 0.6, 1.0)
    # reg_str = trial.suggest_float("regularization_strength", 1e-4, 1e-2, log=True)
    reg_str=1e-3
    pos_weight_val = trial.suggest_float("pos_weight", 1e4, 5e8, log=True)
    # dropout = trial.suggest_float("dropout", 0.1, 0.5)
    dropout=0.3
    pos_weight = torch.tensor([pos_weight_val], device=device)  # Replace 'value' with your desired pos_weight

    num_epochs = 20
    batch_size = 16
    ''' Unused Hyperparameters
    # dropout = trial.suggest_float("dropout", 0.25, 0.5)
    
    # alpha = trial.suggest_float("alpha", 0.25, 1.0)
    # theta = trial.suggest_float("theta", 0.1, 0.9)
    # theta = 0.6
    # gamma = trial.suggest_float("gamma", 2.0, 5.0)
    '''
    
    # ------------------------------ #
    #              DATA              #
    # ------------------------------ #
    aug_params = {
        "patch_size": (104,104,104),
        "final_size":   (104,104,104),
        "flip_prob":  0.5,
        "rot_prob":   0.5,
        "scale_prob": 1.0,
        "rot_range":  np.pi,
        "scale_range": 0.2
    }
    
    train_names, val_names = train_test_split(names, test_size=0.2)

    train_dataset = dataloader.MMapDataset(names=train_names, path=data_path, gpu=True, aug_params=aug_params)
    val_dataset = dataloader.MMapDataset(names=val_names, path=data_path, gpu=True, aug_params=aug_params)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, num_workers=6, collate_fn=dataloader.custom_collate, shuffle=True, pin_memory=True) # can put data aug in collate func later to have optional aug
    val_loader = DataLoader(val_dataset, batch_size=batch_size, num_workers=6, collate_fn=dataloader.custom_collate, shuffle=False, pin_memory=True)
    
    # ------------------------------ #
    #             MODEL              #
    # ------------------------------ #
    model = UNet(
        spatial_dims=3,
        in_channels=1,
        out_channels=1,
        channels=(64, 128, 256, 512),
        strides=(2, 2, 2),
        num_res_units=2,
        dropout=dropout,
    ).to(device)
    

    # ------------------------------ #
    #        TRAINING METHODS        #
    # ------------------------------ #
    bce_loss = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
    def add_regularization_loss(model, regularization_strength):
        reg_loss = 0
        for param in model.parameters():
            reg_loss += torch.sum(param ** 2)
        return regularization_strength * reg_loss
    
    ''' Unused Loss Functions
    # dice_loss = DiceLoss(to_onehot_y=False, softmax=True, weight=weights).to(device)
    # focal_loss = FocalLoss(to_onehot_y=False, use_softmax=True, weight=weights, gamma=gamma ).to(device)
    # mse_loss = nn.MSELoss(reduction='none')
    '''
    
    optimizer = Adam(model.parameters(), lr=lr)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=decay)

    for epoch in range(num_epochs):
    # ------------------------------ #
    #             TRAIN              #
    # ------------------------------ #
        model.train()
        i=0
        batch_loss = 0
        for batch in train_loader:
            input, target = batch['src'].to(device), batch['tgt'].to(device)

            optimizer.zero_grad()
            
            output = model(input)
            loss = bce_loss(output, target)
            reg_loss = add_regularization_loss(model, reg_str)
            loss+=reg_loss

            loss.backward()
            optimizer.step()
            
            # print(f'epoch {epoch} batch {i} loss: {loss.item():.2f}')
            i+=1
            batch_loss+=loss.item()
        print(f'Batch Loss: {(batch_loss / len(train_loader)):.2f}')
        # print(f'Total Time: {total_time:.2f}s, Data Load Time: {(total_time - train_forward_time - train_backward_time):.2f}s, Forward Time: {train_forward_time:.2f}s, Backward Time: {train_backward_time:.2f}s')
        
        scheduler.step()
        
        # ---------- #
        # VALIDATION #
        # ---------- #
        model.eval()
        val_loss = 0
        i=0
        beta = 2.0  # Adjust for F1-score, for example, or set another value for different balance
        precision_metric = BinaryPrecision().to(device)
        recall_metric = BinaryRecall().to(device)
        with torch.no_grad():
            for batch in val_loader:
                input, target = batch['src'].to(device), batch['tgt'].to(device)
                output = model(input)
                
                # Threshold predictions to binary (e.g., 0.5)
                preds = torch.sigmoid(output)  # Apply sigmoid to get probabilities
                preds = (preds > 0.5).float()  # Threshold at 0.5

                # Threshold the target similarly
                target_bin = (target > 0.5).float()  # Convert target to binary based on the same threshold

                # Calculate BCE loss
                # loss = bce_loss(output, target)

                # Update metrics
                precision_metric.update(preds, target_bin)
                recall_metric.update(preds, target_bin)

                # val_loss += loss.item()

            # Compute the final metrics
            precision = precision_metric.compute()
            recall = recall_metric.compute()
            fbeta = (1 + beta**2) * (precision * recall) / (beta**2 * precision + recall)
        if math.isnan(fbeta):
            fbeta = 0.0
        # Print out precision, recall, and F-beta
        print(f"Validation Precision: {precision:.4f}, Recall: {recall:.4f}, F{beta}-score: {fbeta:.4f}")

        # Report to Optuna or any other hyperparameter optimization tool
        # val_loss /= len(val_loader)
        trial.report(fbeta, epoch)
        # print(f"Epoch {epoch} loss: {val_loss:.2f}")
            
        if trial.should_prune():
            
            raise optuna.exceptions.TrialPruned()

    return val_loss

n_epochs = 25

study = optuna.create_study(direction="maximize", pruner=MedianPruner(n_startup_trials=3, n_warmup_steps=10))
study.optimize(objective, n_trials=100)

print("Best hyperparameters:", study.best_params)

[I 2025-03-28 17:41:45,033] A new study created in memory with name: no-name-66d0c2ad-510e-477e-9402-84b72ca24df4


Batch Loss: 45.78
Validation Precision: 0.0000, Recall: 0.7841, F2.0-score: 0.0002
Batch Loss: 54.60
Validation Precision: 0.0005, Recall: 0.8112, F2.0-score: 0.0027
Batch Loss: 39.88
Validation Precision: 0.0000, Recall: 0.8875, F2.0-score: 0.0000
Batch Loss: 19.81
Validation Precision: 0.0006, Recall: 0.8052, F2.0-score: 0.0031
Batch Loss: 59.76
Validation Precision: 0.0007, Recall: 0.8808, F2.0-score: 0.0034
Batch Loss: 48.72
Validation Precision: 0.0006, Recall: 0.9143, F2.0-score: 0.0030
Batch Loss: 20.29
Validation Precision: 0.0013, Recall: 0.8919, F2.0-score: 0.0064
Batch Loss: 39.77
Validation Precision: 0.0001, Recall: 0.8454, F2.0-score: 0.0006
Batch Loss: 81.25
Validation Precision: 0.0015, Recall: 0.9379, F2.0-score: 0.0077
Batch Loss: 18.72
Validation Precision: 0.0011, Recall: 0.9570, F2.0-score: 0.0054
Batch Loss: 19.09
Validation Precision: 0.0006, Recall: 0.8513, F2.0-score: 0.0028
Batch Loss: 56.13
Validation Precision: 0.0000, Recall: 0.0000, F2.0-score: 0.0000
Batc

[I 2025-03-28 18:01:12,160] Trial 0 finished with value: 0.0 and parameters: {'lr': 3.59978294654783e-06, 'pos_weight': 54986.95216082598}. Best is trial 0 with value: 0.0.


Validation Precision: 0.0017, Recall: 0.9346, F2.0-score: 0.0083
Batch Loss: 143096.33
Validation Precision: 0.0000, Recall: 0.3195, F2.0-score: 0.0000
Batch Loss: 267324.78
Validation Precision: 0.0001, Recall: 0.6771, F2.0-score: 0.0004
Batch Loss: 204661.67


[W 2025-03-28 18:04:00,402] Trial 1 failed with parameters: {'lr': 0.00014087364776511183, 'pos_weight': 270106692.55151254} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "c:\Users\hmhor\AppData\Local\Programs\Python\Python310\lib\site-packages\optuna\study\_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
  File "C:\Users\hmhor\AppData\Local\Temp\ipykernel_1192\1934778793.py", line 128, in objective
    for batch in val_loader:
  File "c:\Users\hmhor\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\utils\data\dataloader.py", line 630, in __next__
    data = self._next_data()
  File "c:\Users\hmhor\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\utils\data\dataloader.py", line 1328, in _next_data
    idx, data = self._get_data()
  File "c:\Users\hmhor\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\utils\data\dataloader.py", line 1284, in _get_data
    success, data = 

KeyboardInterrupt: 